# Environment Promotion — Dev → Test → Prod

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LineageLogic/LakeLogic/blob/main/examples/06_advanced_workflows/env_promotion/env_promotion.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/LineageLogic/LakeLogic/blob/main/examples/06_advanced_workflows/env_promotion/env_promotion.ipynb)

---

## 🏢 Business Scenario

Your analytics platform has **40+ data contracts** across Bronze, Silver, and Gold that write to ADLS.
Each environment has its own storage path:

| Environment | ADLS root |
|-------------|--------------------------------------------|
| `dev`       | `abfss://container@account.../dev/…`       |
| `test`      | `abfss://container@account.../test/…`      |
| `prod`      | `abfss://container@account.../prod/…`      |

Without environment promotion, engineers maintain three YAML copies per contract — or run risky
find-and-replace before each deployment.  At 40 contracts that's **120 YAML files** to keep in sync.

## 💡 Value Proposition

- ✅ **One YAML per contract** — no duplication, no drift between environments
- ✅ **Zero code changes on promotion** — set `LAKELOGIC_ENV=test` in CI/CD and you're done
- ✅ **`server.path` is the production default** — no extra config needed for prod
- ✅ **Per-env `format` override** — run CSV locally in dev, Parquet everywhere else
- ✅ **Instant rollback** — revert the env var, every contract re-routes automatically

---

## Goals

1. Understand the `environments` block in a contract
2. Inspect `effective_server()` at each environment level
3. Run Bronze → Silver → Gold locally (simulating `LAKELOGIC_ENV=dev`)
4. See how the same contracts behave when promoted to `test` (ADLS paths)
5. Review quality results and the Gold KPI output


---
## Step 0 — Setup


In [ ]:
import os
import yaml
from pathlib import Path

import polars as pl

from lakelogic import DataProcessor
from lakelogic.core.models import DataContract

# Resolve this example directory regardless of where the notebook is opened
def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        for candidate in [
            base / name,
            base / "examples" / "06_advanced_workflows" / name,
            base / "lakelogic" / "examples" / "06_advanced_workflows" / name,
        ]:
            if candidate.exists():
                return candidate
    return cwd / name

EXAMPLE_DIR = resolve_example_dir("env_promotion")
DATA_DIR    = EXAMPLE_DIR / "data"
CONTRACTS   = EXAMPLE_DIR / "contracts"

# Clean any previous outputs
for f in DATA_DIR.glob("*.parquet"):
    f.unlink()

print(f"Example dir: {EXAMPLE_DIR}")
print(f"Source CSV:  {DATA_DIR / 'orders_raw.csv'}")
print(f"Contracts:   {list(p.name for p in CONTRACTS.glob('*.yaml'))}")


---
## Step 1 — Understand the `environments` block

Each contract has a `server` block (production default) and an `environments` map.
`effective_server(env=...)` merges them — only `path` and `format` are overridable.


In [ ]:
silver_contract = DataContract(**yaml.safe_load((CONTRACTS / "silver_orders.yaml").read_text()))

print("=== silver_orders.yaml ===")
print()
print(f"server.path (prod default):     {silver_contract.server.path}")
print(f"server.format (prod default):   {silver_contract.server.format}")
print()
print("environments block:")
for env_name, env_cfg in silver_contract.environments.items():
    print(f"  [{env_name}]  path={env_cfg.path}  format={env_cfg.format}")
print()

for env in ["dev", "test", None]:
    label = env or "(unset = prod)"
    s = silver_contract.effective_server(env=env)
    print(f"effective_server(env={label!r:20s}) → path={s.path}  format={s.format}")


---
## Step 2 — Run in `dev` mode (local CSV → local Parquet)

Set `LAKELOGIC_ENV=dev` — every contract resolves its local path automatically.
No ADLS credentials needed. This is how engineers run the full pipeline locally.


In [ ]:
os.environ["LAKELOGIC_ENV"] = "dev"
print(f"Active environment: {os.environ['LAKELOGIC_ENV']}")


### Bronze — validate raw CSV orders


In [ ]:
bronze_contract = DataContract(**yaml.safe_load((CONTRACTS / "bronze_orders.yaml").read_text()))
bronze_contract._base_path = CONTRACTS

print(f"Bronze source path (dev): {bronze_contract.effective_server().path}")

processor = DataProcessor(engine="polars", contract=bronze_contract)
result    = processor.run_source(str(DATA_DIR / "orders_raw.csv"))

bronze_good = result.good
bronze_bad  = result.bad

print(f"\n✅ Good rows:  {bronze_good.height}")
print(f"❌ Bad rows:   {bronze_bad.height  if bronze_bad is not None else 0}")

# Persist for Silver to consume
bronze_output = DATA_DIR / "bronze_orders.parquet"
bronze_good.write_parquet(bronze_output)
print(f"\nBronze output written to: {bronze_output}")
bronze_good.head(5)


### Bronze quality report


In [ ]:
report = processor.last_report
print(f"Pass rate:  {report.get('pass_rate', 'n/a')}")
print(f"Good rows:  {report.get('good_rows', 'n/a')}")
print(f"Bad rows:   {report.get('bad_rows', 'n/a')}")
if report.get('rule_results'):
    print("\nRule results:")
    for rule, res in report['rule_results'].items():
        icon = "✅" if res.get('passed') else "❌"
        print(f"  {icon} {rule}")


### Silver — clean, derive `revenue`, enforce quality


In [ ]:
silver_contract._base_path = CONTRACTS

print(f"Silver output path (dev): {silver_contract.effective_server().path}")

processor_s = DataProcessor(engine="polars", contract=silver_contract)
result_s    = processor_s.run_source(str(bronze_output))

silver_good = result_s.good
silver_bad  = result_s.bad

print(f"\n✅ Good rows:  {silver_good.height}")
print(f"❌ Bad rows:   {silver_bad.height if silver_bad is not None else 0}")

silver_output = DATA_DIR / "silver_orders.parquet"
silver_good.write_parquet(silver_output)
print(f"\nSilver output written to: {silver_output}")
silver_good.select(["order_id","customer_id","status","country","revenue"]).head(5)


### Gold — aggregate daily revenue KPI by country + status


In [ ]:
gold_contract = DataContract(**yaml.safe_load((CONTRACTS / "gold_daily_revenue.yaml").read_text()))
gold_contract._base_path = CONTRACTS

print(f"Gold output path (dev): {gold_contract.effective_server().path}")

processor_g = DataProcessor(engine="polars", contract=gold_contract)
result_g    = processor_g.run_source(str(silver_output))

gold_good = result_g.good
print(f"\nGold rows (daily revenue by country+status): {gold_good.height}")

gold_output = DATA_DIR / "gold_daily_revenue.parquet"
gold_good.write_parquet(gold_output)
print(f"Gold output written to: {gold_output}")
gold_good.sort(["order_date","country"])


### 📊 Revenue summary by country


In [ ]:
summary = (
    gold_good
    .group_by("country")
    .agg([
        pl.col("orders").sum().alias("total_orders"),
        pl.col("total_revenue").sum().round(2).alias("total_revenue_gbp"),
        pl.col("avg_order_value").mean().round(2).alias("avg_aov"),
    ])
    .sort("total_revenue_gbp", descending=True)
)
print("Revenue by country (all statuses):")
print(summary)


---
## Step 3 — See how paths switch per environment

The contracts themselves don't change — only `LAKELOGIC_ENV` changes.
This is the core value: **one YAML, any environment**.


In [ ]:
contracts = {
    "bronze_orders":    DataContract(**yaml.safe_load((CONTRACTS / "bronze_orders.yaml").read_text())),
    "silver_orders":    DataContract(**yaml.safe_load((CONTRACTS / "silver_orders.yaml").read_text())),
    "gold_daily_revenue": DataContract(**yaml.safe_load((CONTRACTS / "gold_daily_revenue.yaml").read_text())),
}

print(f"{'Contract':<25}  {'Environment':<12}  {'Effective Path'}")
print("-" * 100)
for env in ["dev", "test", None]:
    label = env if env else "(none=prod)"
    for name, contract in contracts.items():
        s = contract.effective_server(env=env)
        print(f"{name:<25}  {label:<12}  {s.path}")
    print()


---
## Step 4 — CI/CD promotion pattern

The typical CI/CD workflow sets `LAKELOGIC_ENV` as a job-level environment variable.
The cell below shows how it would be called in a GitHub Actions / Azure DevOps step.


In [ ]:
import shutil
import subprocess
import sys

def build_driver_cmd(args):
    exe = shutil.which("lakelogic-driver")
    return [exe] + args if exe else [sys.executable, "-m", "lakelogic.cli.driver"] + args

registry = CONTRACTS / "_registry.yaml"

# ── Dev run (local CSV, no credentials needed) ────────────────────────────────
dev_cmd = build_driver_cmd([
    "--registry", str(registry),
    "--layers", "bronze,silver,gold",
])
print("DEV  →", " LAKELOGIC_ENV=dev ", " ".join(dev_cmd))
print()

# ── Test run (ADLS test paths, test service principal credentials) ───────────
test_cmd = build_driver_cmd([
    "--registry", str(registry),
    "--layers", "bronze,silver,gold",
    "--window", "last_success",
    "--summary-backend", "sqlite",
    "--summary-database", "./run_logs/test_pipeline.db",
])
print("TEST →", " LAKELOGIC_ENV=test", " ".join(test_cmd))
print()

# ── Prod run (ADLS prod paths, LAKELOGIC_ENV unset) ──────────────────────────
prod_cmd = build_driver_cmd([
    "--registry", str(registry),
    "--layers", "bronze,silver,gold",
    "--window", "last_success",
    "--summary-backend", "sqlite",
    "--summary-database", "./run_logs/prod_pipeline.db",
])
print("PROD →", " LAKELOGIC_ENV=(unset)", " ".join(prod_cmd))
print()
print("Only the env var changes — the registry and flags are identical.")


---
## Step 5 — Ad-hoc override (without an environments entry)

For one-off hotfixes or debugging, you can override `server.path` directly via the
`--override` flag — no YAML changes, no environment entry needed.


In [ ]:
hotfix_cmd = build_driver_cmd([
    "--registry", str(registry),
    "--layers", "silver",
    "--override", "server.path=abfss://silver@myaccount.dfs.core.windows.net/hotfix/orders/",
    "--entities", "silver_orders",
])
print("HOTFIX:", " ".join(hotfix_cmd))
print()
print("This overrides the server.path for this single run without touching any YAML.")


---
## Summary

| What you did | How you did it |
|---|---|
| Ran Bronze → Silver → Gold locally | `LAKELOGIC_ENV=dev` → local CSV/Parquet paths |
| Saw how paths switch per env | `contract.effective_server(env=...)` |
| Understood CI/CD promotion | Same registry + flags, different env var |
| Ad-hoc hotfix override | `--override server.path=...` flag |

### Key takeaway

> One YAML file per contract. One env var per environment. Zero changes on promotion.

### Next steps
- `07_production/ci_cd/` — full GitHub Actions deployment pipeline
- `06_advanced_workflows/shared_governance_scale/` — run 500+ contracts in parallel
- `08_compliance_governance/` — GDPR & HIPAA on promoted contracts
